## NOTEBOOK IS NOW DONE

This notebook is now done... 
The database is built, and most queries can be done using SQL language, so I make a new notebook for examples.

In [1]:
cd ..

C:\Users\csav6865\Documents\GitHub\picarro-calculator


In [2]:
import Picarro as pica
import PicarroDatabase as pdb
import pickle

In [3]:
files = pdb.checkforrawdata(r"J:\c715\Picarro\Results\Results 2019\Raw data")

There is no new raw data in directory: J:\c715\Picarro\Results\Results 2019\Raw data


In [4]:
conn = pdb.CreateConnection(r"J:\c715\Picarro\Results\Database\data.db")

#for file in files:
#    pdb.AddRaw(file,conn)
#    try:
#        pdb.AddSummaryRun(file,conn)
#    except:
#        print("An error occurred")

In [5]:
import numpy as np
import glob
import pandas as pd

Here, we've selected the run ids and their respective nicknames for the year 2019

In [6]:
df = pd.read_sql_query("select * from runlookup;",conn)
df

,NickName,RUN_ID
0,2019 01 09 LW test run,20190108
1,2019 01 10 Voegelsberg run1,20190109
2,2019 01 11 Voegelsberg run2,20190110
3,2019 01 12 Voegelsberg run2 rerun,20190111
4,2019 01 13 Voegelsberg run1 rerun,20190112
...,...,...
81,2019 11 19 ControlRun HUSN FALL19,20191119
82,2019 11 20 YD Perm rerun+CS+TP,20191120
83,2019 11 21 GM+reruns+TP+PT,20191121
84,2019 11 22 JP Voegelsberg Run01,20191122


we can also select the runs based on what the name is like, for example: guffert runs.

In [14]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%guffert%';",conn)
df

,NickName,RUN_ID
0,2019 10 23 TR Guffert Run01,20191023
1,2019 11 04 TR Guffert Fall19 Run2,20191104


In [37]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%Hundsalm%';",conn)
df

,NickName,RUN_ID
0,2019 05 14 Hundsalm Run01,20190513
1,2019 05 15 Hundsalm Run02,20190514
2,2019 05 16 Hundsalm Run03,20190515
3,2019 05 17 Hundsalm Run04,20190516
4,2019 07 08 Hundsalm Run05,20190707


Now let's execute a statement where the NickName is replaced.

In [9]:
import sqlite3
from sqlite3 import Error

def ReplaceName(conn,RUN_ID,newname):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param RUN_ID: the run id, an eight digit integer with format yyyymmdd
    :return: updated table name in 

    https://www.sqlitetutorial.net/sqlite-python/create-tables/
    """
    
    statement= """UPDATE runlookup
    SET RUN_ID = {0}, NickName = '{1}'
    WHERE RUN_ID = {0};""".format(RUN_ID,newname)
    
    try:
        c = conn.cursor()

        c.execute(statement)
    except Error as e:
            print(e)
        conn.commit()

In [40]:
ids = [20190513,20190514,20190515,20190516,20190707]
newnames=['2019 05 14 TR Hundsalm Run01',
         '2019 05 15 TR Hundsalm Run02',
         '2019 05 16 TR Hundsalm Run03',
         '2019 05 17 TR Hundsalm Run04',
         '2019 07 08 TR Hundsalm Run05']
for i,j in zip(ids,newnames):
    ReplaceName(conn,i,j)
    



In [41]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%Hundsalm%';",conn)
df

,NickName,RUN_ID
0,2019 05 14 TR Hundsalm Run01,20190513
1,2019 05 15 TR Hundsalm Run02,20190514
2,2019 05 16 TR Hundsalm Run03,20190515
3,2019 05 17 TR Hundsalm Run04,20190516
4,2019 07 08 TR Hundsalm Run05,20190707


In [75]:
pdb.AddSummaryRun(r"J:\c715\Picarro\Results\Results 2019\Raw data\HKDS2009_IsoWater_20190516_142104.csv",conn)

table runs already exists
Running the corrections for Oxygen 
 ... 
 ...
Cannot convert column Analysis to numeric type
Cannot convert column Time Code to numeric type
Cannot convert column Port to numeric type
Cannot convert column Gas Configuration to numeric type
Cannot convert column Method to numeric type


C:\Users\csav6865\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Checking: HU-034-rerun2 ...
too high std. deviation
9701.0
Checking: HU-038-rerun2 ...
Standard dev is good
9703.0
9691.0
Checking: HU-035-rerun2 ...
Standard dev is good
9708.0
Checking: SUR-1 ...
Standard dev is good
9694.0
Checking: DJ-1 ...
Standard dev is good
9696.0
Checking: ADR-1-rerun2 ...
Standard dev too high get rid of measurement 3
9714.0
Checking: HU-022-rerun2 ...
Standard dev is good
9700.0
Checking: HU-017-rerun2 ...
Standard dev is good
9698.0
Checking: HU-021-rerun2 ...
Standard dev too high get rid of measurement 1
9705.0
Checking: TM-ED-03-rerun2 ...
Standard dev is good
9711.0
Checking: HU-029-rerun2 ...
Standard dev is good
9707.0
9690.0
Checking: TM-ED-04-rerun2 ...
Standard dev is good
9712.0
Checking: TM-ED-02-rerun2 ...
Standard dev is good
9710.0
9689.0
9692.0
Checking: TM-ED-05-rerun2 ...
Standard dev is good
9713.0
Checking: HU-018-rerun2 ...
Standard dev is good
9699.0
Checking: HU-036-rerun2 ...
Standard dev is good
9702.0
Checking: TM-ED-Main-rerun2 ...

In [50]:
statement ="""SELECT * from runlookup rlk,runs r 
            WHERE (
            rlk.NickName like '%Hundsalm%'
            AND (r.'Identifier 2' = 'TR' 
            AND r.RUN_ID= rlk.RUN_ID
            ));"""

df = pd.read_sql_query(statement,conn)
df

,NickName,RUN_ID,key,Identifier 1,Identifier 2,RUN_ID,position,d18O vsmow,d18O stdev. vsmow,d18O counts,d2H vsmow,d2H stdev. vsmow,d2H counts,inside GMWL


In [73]:
df = pd.read_sql_query("select r.'Identifier 1',r.'Identifier 2' from rawrun r where RUN_ID = 20190513",conn)

In [76]:
df1 = pd.read_csv(r"J:\c715\Picarro\Results\Results 2019\Raw data\HKDS2009_IsoWater_20190513_142143.csv")

In [77]:
df1.head(50)

,Line,Analysis,Time Code,Port,Inj Nr,d(18_16)Mean,d(D_H)Mean,H2O_Mean,Ignore,Good,...,ch4_ppm,h16od_adjust,h16od_shift,n2_flag,DAS Temp,Tray,Sample,Job,Method,Error Code
0,1,P-9608,2019/05/13 16:27:49,1-01,1,-17.681,-98.989,25012,-1,0,...,2.020,-0.0,-0.0,0,39.812,1,1,1,Picarro pre rinse,0
1,2,P-9608,2019/05/13 16:36:55,1-01,2,-17.388,-94.894,19686,-1,1,...,1.421,0.0,0.0,0,39.938,1,1,1,Picarro pre rinse,0
2,3,P-9608,2019/05/13 16:46:03,1-01,3,-17.172,-93.294,19651,-1,1,...,1.808,0.0,0.0,0,40.000,1,1,1,Picarro pre rinse,0
3,4,P-9608,2019/05/13 16:55:10,1-01,4,-16.834,-92.390,20458,0,1,...,1.709,-0.0,-0.0,0,40.062,1,1,1,Picarro pre rinse,0
4,5,P-9608,2019/05/13 17:04:17,1-01,5,-17.064,-93.069,19772,0,1,...,1.888,-0.0,-0.0,0,40.000,1,1,1,Picarro pre rinse,0
5,6,P-9608,2019/05/13 17:13:25,1-01,6,-16.736,-91.443,20586,0,1,...,2.042,0.0,0.0,0,39.938,1,1,1,Picarro pre rinse,0
6,7,P-9608,2019/05/13 17:22:33,1-01,7,-16.824,-91.826,19479,0,1,...,1.870,-0.0,-0.0,0,39.875,1,1,1,Picarro pre rinse,0
7,8,P-9608,2019/05/13 17:31:41,1-01,8,-16.918,-92.099,18943,0,1,...,1.644,-0.0,-0.0,0,39.875,1,1,1,Picarro pre rinse,0
8,9,P-9608,2019/05/13 17:40:47,1-01,9,-17.026,-92.230,18424,0,1,...,1.633,-0.0,-0.0,0,39.875,1,1,1,Picarro pre rinse,0
9,10,P-9608,2019/05/13 17:49:54,1-01,10,-16.960,-92.242,17846,0,1,...,1.558,-0.0,-0.0,0,39.812,1,1,1,Picarro pre rinse,0


In [87]:
pd.read_sql_query("select * from runlookup rlk,runs r where (rlk.NickName like '%Hundsalm%' and (r.RUN_ID = rlk.RUN_ID and r.'Identifier 2'  like '%Hundsalm%'));",conn)

,NickName,RUN_ID,key,Identifier 1,Identifier 2,RUN_ID,position,d18O vsmow,d18O stdev. vsmow,d18O counts,d2H vsmow,d2H stdev. vsmow,d2H counts,inside GMWL
0,2019 05 15 TR Hundsalm Run02,20190514,9640,HU-020,190511Hundsalm,20190514,6,-10.710960,0.046163,2,-75.519442,0.000470,2,inside
1,2019 05 15 TR Hundsalm Run02,20190514,9641,HU-021,190511Hundsalm,20190514,7,-10.066203,0.056920,3,-70.283194,0.306929,3,inside
2,2019 05 15 TR Hundsalm Run02,20190514,9642,HU-022,190511Hundsalm,20190514,8,-10.388315,0.024255,2,-73.110994,0.465252,2,inside
3,2019 05 15 TR Hundsalm Run02,20190514,9644,HU-023,190511Hundsalm,20190514,10,-10.128032,NaN,1,-71.225339,NaN,1,inside
4,2019 05 15 TR Hundsalm Run02,20190514,9645,HU-024,190511Hundsalm,20190514,11,-10.144330,NaN,1,-71.085296,NaN,1,inside
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2019 07 08 TR Hundsalm Run05,20190707,9899,HU-045,20190601Hundsalm,20190707,22,-12.655751,0.021190,4,-91.119090,0.119674,4,inside
60,2019 07 08 TR Hundsalm Run05,20190707,9900,HU-042-7DAY,20190601Hundsalm,20190707,23,-12.962829,0.019094,4,-95.576305,0.234611,4,inside
61,2019 07 08 TR Hundsalm Run05,20190707,9901,HU-043-7DAY,20190601Hundsalm,20190707,24,-12.736365,0.035587,4,-91.859017,0.197248,4,inside
62,2019 07 08 TR Hundsalm Run05,20190707,9902,XX4-7DAY,20190601Hundsalm,20190707,25,-9.190924,0.080785,4,-64.388225,0.472473,4,inside


In [ ]:
pd.read_sql_query("select * from runs where")

In [90]:
pd.read_sql_query("select * from runs",conn)

,key,Identifier 1,Identifier 2,RUN_ID,position,d18O vsmow,d18O stdev. vsmow,d18O counts,d2H vsmow,d2H stdev. vsmow,d2H counts,inside GMWL
0,9095.0,TAP,_Conditioning,20190108,1.0,-13.561595,0.071819,4,-94.428744,0.398818,4,inside
1,9096.0,HAUS1,_Standard,20190108,2.0,0.423282,0.023795,10,3.127900,0.903848,10,outside
2,9097.0,HAUS2,_Standard,20190108,3.0,-30.016903,0.197028,10,-230.182077,1.744176,10,inside
3,9098.0,TAP,_Standard,20190108,4.0,-13.086380,0.194695,10,-94.245823,0.573695,10,inside
4,9099.0,W22,_Control W22,20190108,5.0,-3.222449,0.103634,4,-21.246801,1.004746,4,inside
...,...,...,...,...,...,...,...,...,...,...,...,...
2045,11441.0,VB035,JP,20191123,23.0,-11.711361,0.056990,4,-81.962680,0.373372,4,inside
2046,11442.0,VB036,JP,20191123,24.0,-11.196166,0.091225,4,-78.521708,0.592135,4,inside
2047,11443.0,VB037,JP,20191123,25.0,-10.733680,0.099246,4,-74.051553,0.327689,4,inside
2048,11444.0,VB038,JP,20191123,26.0,-11.884129,0.030368,4,-85.096818,0.229076,4,inside
